In [ ]:
# Install detectron2
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html
!pip install pyyaml==5.1
!pip install torch==1.7.0
!pip install torchvision==0.8.1
exit(0)  

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html
     |████████████████████████████████| 6.8MB 24.4MB/s 
     |████████████████████████████████| 2.2MB 14.3MB/s 
     |████████████████████████████████| 51kB 7.8MB/s 
     |████████████████████████████████| 645kB 50.0MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.3.post20210311-cp37-none-any.whl size=58461 sha256=b0642a584761356fc45e6272a911f343dab45c27d49d926a3a448b7fd228ecb9
  Stored in directory: /root/.cache/pip/wheels/bc/cc/93/e011d6b86d91631d948f9bda94d60b82b51302192eb8a688c8
  Created wheel for iopath: filename=iopath-0.1.6-cp37-none-any.whl size=18269 sha256=17827bed251568566f1fad94114dfc5bda47d069e5d7d335e3a67ba337a7d524
  Stored in directory: /root/.cache/pip/wheels/ff/87/55/7513122394822977f993e1798bfebb46730d6183df8db7b283
Successfully built fvcore iopath
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Fo

     |████████████████████████████████| 276kB 10.4MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44074 sha256=a25b09960e6995ea0aaea06978e7d53365c3c8e1e21cb6595752427932cb19fd
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 5.4.1
    Uninstalling PyYAML-5.4.1:
      Successfully uninstalled PyYAML-5.4.1
     |████████████████████████████████| 776.8MB 23kB/s 
ERROR: torchvision 0.9.0+cu101 has requirement torch==1.8.0, but you'll have torch 1.7.0 which is incompatible.
ERROR: torchtext 0.9.0 has requirement torch==1.8.0, but you'll have torch 1.7.0 which is incompatible.
  Found existing installation: torch 1.8.0+cu101
    Uninstalling torch-1.8.0+cu101:
      Successfully uninstalled torch-1.8.0+cu101
     |████████████████████████████████| 12.7MB 9.5MB/s 
  Found existing installation: torchvision 0.9.0+cu101
    Uninstalling

In [ ]:
# Install timer
!pip install ipython-autotime
%load_ext autotime

time: 1.34 ms (started: 2021-03-11 22:08:06 +00:00)


In [ ]:
!pip install -U iopath

Requirement already up-to-date: iopath in /usr/local/lib/python3.7/dist-packages (0.1.6)
time: 2.23 s (started: 2021-03-11 22:08:12 +00:00)


In [ ]:
# Mount drive

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive
time: 16.7 s (started: 2021-03-11 22:08:14 +00:00)


In [ ]:
# Check torch version == 1.7
import torch
assert torch.__version__.startswith("1.7")

time: 398 ms (started: 2021-03-11 22:08:32 +00:00)


In [ ]:
# Import human detector/detectron utils
import glob
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import numpy as np
import os, json, cv2, random, math

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

# Import pose estimation utils

import tensorflow as tf
from tensorflow.keras import Model, layers, models, optimizers
from IPython.display import clear_output 

import skimage.io as io
from skimage.transform import resize
from google.colab import files
from timeit import default_timer as timer

import skimage.io as io
import matplotlib.pyplot as plt
import pylab
from google.colab.patches import cv2_imshow



from IPython.display import HTML
from base64 import b64encode


** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 



time: 3.35 s (started: 2021-03-11 22:08:38 +00:00)


In [ ]:
########## functions
# load human detector
def load_human_detector(detect_threshold):
  cfg = get_cfg()
  cfg.merge_from_file(model_zoo.get_config_file("Misc/mask_rcnn_R_50_FPN_1x_dconv_c3-c5.yaml"))
  cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = detect_threshold  # set threshold for this model
  cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("Misc/mask_rcnn_R_50_FPN_1x_dconv_c3-c5.yaml")
  temp_detector = DefaultPredictor(cfg)

  return temp_detector

# load pose estimation model
def load_pose_es(weight_dir):
  def build_model():
    base_model = tf.keras.applications.ResNet152(include_top=False, weights = None, input_shape = (256,192,3))
    # base_model = tf.keras.applications.ResNet50(include_top=False, weights ='imagenet', input_shape = (256,192,3))
    # base_model = tf.keras.applications.MobileNetV2(include_top=False, weights = None, input_shape = (256,192,3))
    xxx = base_model.output
    xxx = layers.Conv2DTranspose(filters= 256,kernel_size= [4,4], strides= 2, padding= 'same', kernel_initializer= 'TruncatedNormal',activation='relu')(xxx)
    xxx = layers.Conv2DTranspose(filters= 256,kernel_size= [4,4], strides= 2, padding= 'same', kernel_initializer= 'TruncatedNormal',activation='relu')(xxx)
    xxx = layers.Conv2DTranspose(filters= 256,kernel_size= [4,4], strides= 2, padding= 'same', kernel_initializer= 'TruncatedNormal',activation='relu')(xxx)
    xxx = layers.Conv2D(filters= 17,kernel_size= [1,1], padding= 'same')(xxx)
    model_temp = models.Model(inputs=base_model.input, outputs=xxx)
    return model_temp
  temp_model = model = build_model()
  temp_model.load_weights(weight_dir)

  return temp_model

# get boundary boxes from images:
def img_to_bboxes(img_o,predictor):
  temp_output = predictor(img_o)
  temp_ints = temp_output['instances']
  temp_ints = temp_ints[temp_ints.pred_classes == 0]
  n_ints = len(temp_ints)
  img_h,img_w = temp_ints.image_size
  out_bboxes = []
  bboxes = [temp_ints.pred_boxes[x].tensor.tolist()[0] for x in range(n_ints)]

  for i in range(len(bboxes)):

    i_bbox = bboxes[i]
    bbox_x, bbox_y,bbox_x2,bbox_y2 = i_bbox
    bbox_w = bbox_x2-bbox_x
    bbox_h = bbox_y2-bbox_y
    to_check = 0.75*bbox_h
    if to_check >= bbox_w:
      add_x = True
    else:
      add_x = False

    if add_x:
      new_bbox_h = bbox_h
      new_bbox_w = 0.75*bbox_h
      diff = new_bbox_w - bbox_w
      new_bbox_y = bbox_y
      new_bbox_x = bbox_x - 0.5*diff
      #check if in image
      if new_bbox_x < 0:
        new_bbox_x = 0
      if new_bbox_x+new_bbox_w >= img_w:
        new_bbox_x = img_w - new_bbox_w - 1
    else:
      new_bbox_w = bbox_w
      new_bbox_h = 4.0/3.0 * bbox_w
      diff = new_bbox_h - bbox_h
      new_bbox_x = bbox_x
      new_bbox_y = bbox_y - 0.5 * diff
      if new_bbox_y < 0:
        new_bbox_y = 0
      if new_bbox_y+new_bbox_h >= img_h:
        new_bbox_y = img_h - new_bbox_h - 1

    temp_new_bbox = [new_bbox_x,new_bbox_y,new_bbox_w,new_bbox_h]
    # print(temp_new_bbox)
    out_bboxes.append(temp_new_bbox)

  return out_bboxes

# Decode heatmaps to joints
def heatmaps_to_joints(i_heatmaps):
  o_J = []
  n_h = p_heatmaps.shape[0]
  for k in range(n_h):
    temp_heatmap = i_heatmaps[k,:,:,:]
    tkp_x = []
    tkp_y = []
    for j in range(17):
      temp_max = np.where(temp_heatmap[:,:,j] == np.amax(temp_heatmap[:,:,j]))
      ty = temp_max[0][0]
      tx = temp_max[1][0]

      tkp_x.append(tx)
      tkp_y.append(ty)

    tkp = [tkp_x,tkp_y]
  
    tkp = np.asarray(tkp)
    o_J.append(tkp)
  return o_J

# Create new pose
def new_Pose(nf=None,iid=None,jts=None,ibbox=None):
  t_new_pose = Pose()
  if nf is not None:
    t_new_pose.frame = nf
  if iid is not None:
    t_new_pose.id = iid
  if jts is not None:
    t_new_pose.joints = jts
  if ibbox is not None:
    t_new_pose.bbox = ibbox
  
  return t_new_pose

# Calculate similarity matric from OKS
def cal_sim_mat(input_Q,input_Ji):

  OKS_k = np.array([0.026,0.025,0.025,0.035,0.035,0.079,0.079,0.072,0.072,0.062,0.062,0.107,0.107,0.087,0.087,0.089,0.089])
  n_Q = len(input_Q)
  n_J = len(input_Ji)
  sim_mat = np.full([n_J, n_Q], None)
  for ij in range(n_J):
    for iq in range(n_Q):
      i_joint = input_Ji[ij] 
      j_P = input_Q[iq]
      tx1,tx2,tw,th = j_P.bbox
      s2 = tw*th
      j_joint = j_P.joints
      # calculate score
      d2 = (i_joint - j_joint)**2
      d2 = np.sum(d2,axis=0)
      term =  np.divide(d2,OKS_k**2)
      term = term/(2*s2)
      term_exp = np.exp(-1*term)
      temp_OKS = np.sum(term_exp)/17
      # assign score to sim_mat
      sim_mat[ij,iq] = temp_OKS

  return sim_mat

# get id list to assign

def get_id_to_assign(inputo_pose_id,input2_Q,input2_Ji,input_sim_mat):
  get_id_sim_mat = input_sim_mat
  input_pose_id = inputo_pose_id
  input_nh = len(input2_Ji)

  can_old_id = list(range(input_pose_id))
  can_new_id = list(range(input_nh))
  id_to_assign = np.full([1,input_nh],None)

  max_score_th = 0.8

  while len(can_old_id)>0 and len(can_new_id)>0:
    # print(get_id_sim_mat)
    # print(id_to_assign)
    max_score = np.amax(get_id_sim_mat)
    if max_score_th < 0.8:
      break
    else:  
      res = np.where(get_id_sim_mat == np.amax(get_id_sim_mat))
      max_pos = list(zip(res[0], res[1]))[0]
      mi,mj = max_pos
      m_P = input2_Q[mj]
      m_id = m_P.id
      id_to_assign[0,mi] = m_id
      # print(m_id)
      can_old_id.remove(m_id)
      can_new_id.remove(mi)
      column_to_rm = [x for x in range(len(input2_Q)) if input2_Q[x].id == m_id]
      row_to_rm = mi
      get_id_sim_mat[:,column_to_rm] = 0
      get_id_sim_mat[row_to_rm,:] = 0

  n_new = np.sum((id_to_assign == None).astype('int'))
  if n_new > 0:
    for l in range(input_nh):
      id_to_check = id_to_assign[0,l]
      if id_to_check == None:
        id_to_assign[0,l] = input_pose_id
        input_pose_id = input_pose_id + 1
  id_to_assign = id_to_assign.tolist()[0]

  return input_pose_id, id_to_assign



time: 351 ms (started: 2021-03-11 22:08:44 +00:00)


In [ ]:
class Pose:
  def __init__(self):
    self.frame = None
    self.id = None
    self.joints = None
    self.bbox = None

time: 2.43 ms (started: 2021-03-11 22:08:48 +00:00)


In [ ]:
# my_cfg

human_detector_threshold = 0.9

# coco
# pose_es_weight_dir = './gdrive/MyDrive/posetrack_demo/mymodel/best_weight152'

# posetrack
pose_es_weight_dir = './gdrive/MyDrive/posetrack_demo/mymodel/best_weight_posetrack_2'


# images_dir = '/content/gdrive/MyDrive/posetrack_demo/demo_images_track/000003_bonn_train'
# images_dir = '/content/gdrive/MyDrive/posetrack_demo/demo_images_track/001001_mpii_test'
images_dir = '/content/gdrive/MyDrive/posetrack_demo/demo_images_track/009607_mpii_test'
# images_dir = '/content/gdrive/MyDrive/posetrack_demo/demo_images_track/015302_mpii_test'

# pose es input/output
input_shape = (256, 192)
output_shape = (64, 48)


time: 5.59 ms (started: 2021-03-11 22:14:12 +00:00)


In [ ]:
# set up

# load human detector

hdetector = load_human_detector(human_detector_threshold)

# load pose estimator

pose_estimator = load_pose_es(pose_es_weight_dir)



time: 8.79 s (started: 2021-03-11 22:14:14 +00:00)


In [ ]:
# pipeline

# delete mp4
for filename in sorted(glob.glob('./*.mp4')):
  os.remove(filename)

Q = []
frame_id = 0
pose_id = 0
for filename in sorted(glob.glob(images_dir+'/*.jpg')):
  
  img = io.imread(filename)
  temp_bboxes = img_to_bboxes(img,hdetector)
  batch_imgs = []
  for i_bbox in temp_bboxes:
    o_crop = img[int(i_bbox[1]):int(i_bbox[1]+i_bbox[3]),int(i_bbox[0]):int(i_bbox[0]+i_bbox[2]),:]
    if o_crop.shape[0] == 0 or o_crop.shape[1]  == 0 or o_crop.shape[2] == 0:
      print('detect empty image')
      continue
    o_crop = resize(o_crop,input_shape)
    o_crop = o_crop.astype('float32')
    batch_imgs.append(o_crop)
  nh = len(batch_imgs)
  batch_imgs = np.array(batch_imgs)
  batch_imgs = tf.convert_to_tensor(batch_imgs,dtype=tf.float32)
  p_heatmaps = pose_estimator.predict(batch_imgs)
  temp_Ji = heatmaps_to_joints(p_heatmaps)
  if frame_id == 0:
    for j in range(nh):
      temp_bbox = temp_bboxes[j]
      temp_j = temp_Ji[j]
      temp_pose = new_Pose(frame_id,pose_id,temp_j,temp_bbox)
      Q.append(temp_pose)
      pose_id = pose_id + 1
    frame_id = frame_id + 1
    
  else:
    # similarity matrix from Joints
    temp_sim_mat = cal_sim_mat(Q,temp_Ji)
    # print(temp_sim_mat)
    # Assign id and create Pose 
    pose_id, temp_ids = get_id_to_assign(pose_id,Q,temp_Ji,temp_sim_mat)
    # print(temp_ids)
    # print('-----------------------------------')
    for j in range(nh):
      temp_bbox = temp_bboxes[j]
      # print(temp_bbox)
      temp_j = temp_Ji[j]
      t_pose_id = temp_ids[j]
      temp_pose = new_Pose(frame_id,t_pose_id,temp_j,temp_bbox)
      # print(temp_pose.bbox)
      Q.append(temp_pose)
    frame_id = frame_id + 1




time: 1min 9s (started: 2021-03-11 22:14:22 +00:00)


In [ ]:
# frame_count = 0
# for q in Q:
#   if q.frame == frame_count:
#     print('-------------------------------')
#     frame_count = frame_count + 1
#   print('frame: {}'.format(q.frame))
#   print('id: {}'.format(q.id))
#   print(q.bbox)

time: 1.59 ms (started: 2021-03-11 22:15:32 +00:00)


In [ ]:
# visualize!
img_array = []
ori_array = []
frame_count = 0

img = io.imread(filename)
i_h,i_w,i_l = img.shape
img_size = (i_w,i_h)

for filename in sorted(glob.glob(images_dir+'/*.jpg')):
  img = cv2.imread(filename)
  ori_array.append(img)
  check_list = [x for x in range(len(Q)) if Q[x].frame == frame_count]
  Qs = []
  for cl in check_list:
    Qs.append(Q[cl])

  for qs in Qs:
    tid = qs.id
    # print(tid)
    tbbox = qs.bbox
    tkps = qs.joints

    o_kps = np.transpose(tkps)
    
    # scale kps to image size
    if tbbox[2] == 0:
      tbbox[2] = 1
    if tbbox[3] == 0:
      tbbox[3] = 1
    # print('from ({},{}) to ({},{})'.format(output_shape[1],output_shape[0],tbbox[2],tbbox[3]))
    scale_x = tbbox[2]/output_shape[1]
    scale_y = tbbox[3]/output_shape[0]

    temp_scale = np.array([scale_x,scale_y])
    for ikps in range(len(o_kps)):
      temp_val = o_kps[ikps,:]
      # print('b4_scale: {}'.format(temp_val))
      o_kps[ikps,:] = np.multiply(temp_val,temp_scale)
      # print('at_scale: {}'.format(o_kps[ikps,:]))
    # convert kps to image location(global)
    g_kps = o_kps
    o_x = tbbox[0]
    o_y = tbbox[1]
    o_origin = np.array([tbbox[0],tbbox[1]])
    for gkps in range(len(g_kps)):
      if g_kps[gkps,0] != 0:
        g_kps[gkps,:] = g_kps[gkps,:] + o_origin
    # draw bbox
    c_code = (0,255,0)
    
    cv2.rectangle(img,(int(tbbox[0]),int(tbbox[1])),(int(tbbox[0]+tbbox[2]),int(tbbox[1]+tbbox[3])),c_code,3)
    # put id in image

    pos_txt = (int(tbbox[0]+tbbox[2]-50),int(tbbox[1]+tbbox[3]-10))
    cv2.putText(img,str(qs.id),pos_txt,cv2.FONT_HERSHEY_COMPLEX,2,c_code,thickness=3)
    # draw skeleton
    skeleton_list = [(0,1),(0,2),(0,3),(0,4),(1,5),(1,6),(5,7),(6,8),(7,9),(8,10),(6,12),(5,11),(12,11),(12,14),(11,13),(14,16),(13,15)]
    # line
    for sk in skeleton_list:
      p1,p2 = sk
      x1,y1 = g_kps[p1,:].tolist()
      x2,y2 = g_kps[p2,:].tolist()
      cv2.line(img,(x1,y1),(x2,y2),(0,255,0),3)
    # dot
    for i in range(17):
      x,y = g_kps[i,:].tolist()
      cv2.circle(img,(x,y),2,(0,0,255),3)

  img_array.append(img)
  frame_count = frame_count + 1



out = cv2.VideoWriter('original.mp4',cv2.VideoWriter_fourcc(*'MP4V'), 15, img_size)
for i in range(len(ori_array)):
    out.write(ori_array[i])
out.release()

out = cv2.VideoWriter('output.mp4',cv2.VideoWriter_fourcc(*'MP4V'), 15, img_size)
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()


time: 3.73 s (started: 2021-03-11 22:15:32 +00:00)


In [ ]:
# Input video path
save_path = "/content/output.mp4"

# Compressed video path
compressed_path = "/content/result_to_show.mp4"

os.system(f"ffmpeg -i {save_path} -vcodec libx264 {compressed_path}")

# Show video
mp4 = open(compressed_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=1000 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

time: 7.73 s (started: 2021-03-11 22:15:36 +00:00)
